In [1]:
import numpy as np
import pandas as pd
import os
import torch
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
access_token = os.getenv('HUGGINGFACE_TOKEN')

In [3]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B", token=access_token)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B", token=access_token)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
print(torch.cuda.get_device_name(torch.cuda.current_device()))

In [4]:
# Move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define a function to generate text
def generate_text(prompt):
    inputs = tokenizer.encode_plus(
        prompt,
        add_special_tokens=True,
        return_attention_mask=True,
        max_length=100000,
        return_tensors='pt'
    )

    # Move the input tensors to the GPU
    inputs['input_ids'] = inputs['input_ids'].to(device)
    inputs['attention_mask'] = inputs['attention_mask'].to(device)

    outputs = model.generate(
        inputs['input_ids'],
        max_length=100000,
        attention_mask=inputs['attention_mask']
        
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [ ]:
df = pd.read_csv("../Datasets/Attractions.csv", header = None)
text0 = df.to_string()
print(len(text0))

df = pd.read_csv("../Datasets/Hotels.csv", header = None)
text1 = df.to_string()
print(len(text1))

df = pd.read_csv("../Datasets/Restaurants.csv", header = None)
text2 = df.to_string()
print(len(text2))

In [ ]:
backgroud = text0 + text1 + text2
with open('background_all.txt', 'w') as f:
    f.write(backgroud)

In [5]:
with open('background_all.txt', 'r') as f:
    prompt_background = f.read()

In [ ]:
prompt_background

In [ ]:
# Test the function
prompt = "Give me a travel plan in Philadelphia for 2 days, given the information of the following business" + prompt_background
generated_text = generate_text(prompt)
print(generated_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
